In [1]:
import seaborn
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib import pylab as plt
import root_numpy
import pandas as pd


BRICK_X = 124000
BRICK_Y = 99000
BRICK_Z = 75000
SAFE_M = 25000

/Users/artem/anaconda/envs/py27/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/artem/anaconda/envs/py27/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### test & burn

### end of tests

In [2]:
def load_mc(filename="mcdata2.root", step=1):
    mc = root_numpy.root2array(
                        filename,
                        treename="Data",
                        branches=["Event_id", "ele_P",
                                   "BT_X", "BT_Y", "BT_Z",
                                   "BT_SX", "BT_SY",
                                   "ele_x", "ele_y", "ele_z", 
                                   "ele_sx", "ele_sy", 
                                   "chisquare"],
                        step=step)
    pmc = pd.DataFrame(mc)
    pmc['numtracks'] = pmc.BT_X.apply(lambda x: len(x))
    # cuts
    shapechange = [pmc.shape[0]]
    pmc = pmc[pmc.ele_P > 0.1]
    shapechange.append(pmc.shape[0])
    pmc = pmc[pmc.ele_x.abs() < BRICK_X / 2 - SAFE_M]
    shapechange.append(pmc.shape[0])
    pmc = pmc[pmc.ele_y.abs() < BRICK_Y / 2 - SAFE_M]
    shapechange.append(pmc.shape[0])
    pmc = pmc[pmc.ele_z < 0]
    shapechange.append(pmc.shape[0])
    pmc = pmc[pmc.numtracks > 3]
    shapechange.append(pmc.shape[0])
    print "numtracks reduction by cuts: ", shapechange
    # calc centers
    # to brick reference frame and center
    pmc['m_BT_X'] = pmc.BT_X.apply(lambda x: x.mean())
    pmc['m_BT_Y'] = pmc.BT_Y.apply(lambda x: x.mean())
    pmc['m_BT_Z'] = pmc.BT_Z.apply(lambda x: x.mean())
    pmc.BT_X += BRICK_X / 2
    pmc.BT_Y += BRICK_Y / 2
    pmc.BT_Z += BRICK_Z / 2
    pmc.ele_x += BRICK_X / 2
    pmc.ele_y += BRICK_Y / 2
    pmc.ele_z += BRICK_Z / 2
    #print "len(pmc): {len}".format(len=len(pmc))
    return pmc
    

In [3]:
def plot_ith_quiver(mc, id=0):
    z = mc[id]
    dZ = 30
    U = z["BT_SX"] * dZ
    V = z["BT_SY"] * dZ
    W = dZ
    fig = plt.figure(figsize=(10,6))
    ax = fig.gca(projection='3d')
    _ = ax.quiver(z["BT_X"], z["BT_Y"], z["BT_Z"], U, V, W, length=100)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    print "electron momentum: {p:.2f}, len(slopes): {slopes}".format(p=z['ele_P'], slopes=len(U))

In [13]:
def plot_mc(mcdf, id=0):
    dZ = 205
    df = mcdf.iloc[id]
    # mind the order!
    d0 = pd.DataFrame([
                df['BT_Z'],
                df['BT_X'],
                df['BT_Y']],
                index=['z', 'x', 'y']).T
    numtracks = d0.shape[0]
    dd = pd.DataFrame([
            df['BT_SX']*dZ,
            df['BT_SY']*dZ],
            index=['x', 'y']).T
    dd.insert(loc=0, column='z', value=dZ)
    d1 = d0 + dd
    # print d0, d1
    C = plt.cm.Reds(0.5)
    lc = Line3DCollection(zip(d0.values, d1.values), colors=C, alpha=0.9, lw=2)

    fig = plt.figure(figsize=(12,8))
    ax = fig.gca(projection='3d')
    ax.view_init(azim=-103, elev=16)
    ax.add_collection3d(lc)
    
    # mind the order!
    ax.set_xlabel("z")
    ax.set_ylabel("x")
    ax.set_zlabel("y")  
    ax.set_xlim(d0['z'].min(), d0['z'].max())
    ax.set_ylim(d0['x'].min(), d0['x'].max())
    ax.set_zlim(d0['y'].min(), d0['y'].max())
    print "len(slopes): {slopes}".format(slopes=numtracks)

In [14]:
%matplotlib qt
mc = load_mc(step=100)
plot_mc(mc, id=0)

numtracks reduction by cuts:  [188, 186, 109, 50, 19, 19]
len(slopes): 731


In [7]:
def meta_mc_analyse(mc):
    #_ = plt.hist(mc['ele_P'], bins=100, range=[0, 5])
    lens = map(lambda x: len(x[1].BT_X), mc.iterrows())
    _ = plt.hist(lens, bins=100, range=(0,20))
    p = pd.DataFrame(lens)
    #print p
    print p[p[0] < 3]